In [21]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data
from PIL import Image
from torch.utils.data.dataset import Dataset
import numpy as np
import pandas as pd

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pickle

In [22]:
num_classes = 120
batch_size = 5
epochs = 15
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [23]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize])
valid_transform = transforms.Compose([transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize])


trainset = torchvision.datasets.ImageFolder("./data/train/", train_transform);
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

validset = torchvision.datasets.ImageFolder("./data/valid/", valid_transform);
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

classes = trainloader.dataset.classes
pickle.dump(classes, open("dog_breeds_labels.pickle", "wb"), 2)

In [24]:
def train(model, train, valid, optimizer, criterion, epochs=1):
    for epoch in range(epochs):
        print('Epoch ', epoch + 1, '/', epochs)
        
        running_loss = 0.
        running_corrects = 0.
        running_batches = 0.
       
        model.train()
        for i, (input, target) in enumerate(train):
            input_var = torch.autograd.Variable(input)
            target_var = torch.autograd.Variable(target)

            optimizer.zero_grad()

            output = model(input_var)
            _, preds = torch.max(output.data, 1)
            loss = criterion(output, target_var)

            loss.backward()
            optimizer.step()
            
            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == target)
            running_batches += 1.
            
            ''' for testing 
            if i > 10:
                break
            '''

            print('\r', 'Batch', i, 'Loss', loss.data[0], end='')
            
        train_loss = running_loss / running_batches
        train_acc = running_corrects / len(train.dataset)
        print('\r', "Train Loss", train_loss, "Train Accuracy", train_acc)
            
        running_loss = 0.
        running_corrects = 0.
        running_batches = 0.

        model.eval()
        for i, (input, target) in enumerate(valid):
            input_var = torch.autograd.Variable(input, volatile=True)
            target_var = torch.autograd.Variable(target, volatile=True)

            output = model(input_var)
            _, preds = torch.max(output.data, 1)
            loss = criterion(output, target_var)

            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == target)
            running_batches += 1.
            
            ''' for testing 
            if i > 10:
                break
            '''

        valid_loss = running_loss / running_batches
        valid_acc = running_corrects / len(valid.dataset)
        print('\r', "Val Loss", valid_loss, "Val Accuracy", valid_acc)

In [25]:
model = torchvision.models.alexnet(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
num_features = model.classifier[6].in_features
features = list(model.classifier.children())[:-1]
features.extend([nn.Linear(num_features, num_classes)])
#replace the classifier of the trained network with our dog classifier
#parameters of newly constructed modules have required_grad=True by default
#model.fc = nn.Linear(num_features, num_classes) 

model.classifier = nn.Sequential(*features)
for param in model.classifier.parameters():
    param.requires_grad = True
    
criterion = nn.CrossEntropyLoss()
#Optimize only the classifier
optimizer = torch.optim.SGD(list(filter(lambda p: p.requires_grad, model.parameters())), lr=0.001, momentum=0.9)

train(model, trainloader, validloader, optimizer, criterion, epochs=epochs)

Epoch  1 / 15
 Train Loss 3.0162620454392535 Train Accuracy 0.286948059846734
 Val Loss 2.0276138120728016 Val Accuracy 0.456271864067966
Epoch  2 / 15
 Train Loss 2.4084897257002655 Train Accuracy 0.3904634472691887
 Val Loss 1.9403785601458738 Val Accuracy 0.4752623688155922
Epoch  3 / 15
 Train Loss 2.2282566048884282 Train Accuracy 0.4353484977496655
 Val Loss 1.9679940441684876 Val Accuracy 0.4877561219390305
Epoch  4 / 15
 Train Loss 2.1597433317577224 Train Accuracy 0.4465393504439849
 Val Loss 1.9807465990489705 Val Accuracy 0.4702648675662169
Epoch  5 / 15
 Train Loss 2.079471355142202 Train Accuracy 0.46733973969103515
 Val Loss 2.0347287806942407 Val Accuracy 0.45077461269365315
Epoch  6 / 15
 Train Loss 2.0823272359905634 Train Accuracy 0.4634472691886632
 Val Loss 2.08511346311975 Val Accuracy 0.4577711144427786
Epoch  7 / 15
 Train Loss 2.0326897098391252 Train Accuracy 0.47621943802457123
 Val Loss 1.965133282407059 Val Accuracy 0.49375312343828087
Epoch  8 / 15
 Train L

In [26]:
torch.save(model.state_dict(), 'dog-breed-ident-AlexNet.pt')

In [27]:
class DogsData(Dataset):
    def __init__(self, root_dir, labels, transform, output_class=True):
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.output_class = output_class
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = self.labels.iloc[idx]
        path = '{}/{}.jpg'.format(self.root_dir, item['id'])
        image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)
            
        if not self.output_class:
            return image
        
        return image, item['class']

test_loader = torch.utils.data.DataLoader(
    DogsData('./data/test/x', sample_submission, valid_transform, output_class=False),
    batch_size=batch_size
)

In [28]:
model.load_state_dict(torch.load('dog-breed-ident-AlexNet.pt'))
model.eval()
results = []

for i, input in enumerate(test_loader):
    input_var = torch.autograd.Variable(input, volatile=True)
    output = model(input_var)
    results.append(F.softmax(output, dim=1).cpu().data.numpy())
    print('\r', 'Batch', i, end='')
    
    ''' for testing 
    if i > 10:
        break
    '''
        
results = np.concatenate(results)

 Batch 2071

In [29]:
results.shape

(10357, 120)

In [30]:
ids = sample_submission['id'].values
sample_df = pd.DataFrame(ids, columns=['id'])
#sample_df = sample_df[:48] #for testing only
#sample_df.shape
for index, breed in enumerate(classes):
    sample_df[breed] = results[:,index]

In [31]:
sample_df.to_csv('pred_AlexNet.csv', index=None)